In [1]:
from bs4 import BeautifulSoup
import re
import pandas as pd

# INTRUCCIONES

Este Scrapper funciona de forma más manual, y con cierta intervención.

Formatea campos basicos, como ubicación, paga, etc. Lo que deja algunos campos vacios, que deben de ser analizados manualmente.

En cierta forma sigue siendo mejor que ahcer todo desde 0

---

# 1 Acceder a Indeed

Esta aprte es Full Manual, ya que Indeed tiene una politica MUY PESADA contra Scrips, lo que vuelve tediosa su automatización.

[INDEED](https://www.indeed.com)

Y buscar el puesto de trabajo solicitado.

# 2 Descargar HTML de la página.

**Control + s**

Preferentemente descargar con el nombre *jobs.html* despues de eso, subir a la carpeta actual.

# 3 Ejecutar celdas

Con todo lo anterior realizado, se debe de crear al final un bonito EXCEL con la mayoria de los campos correctamente formateados.

# 4 Skills.

Este campo no se parse Bien (Completamente) ya que unicamente refleja el fragmento disponible del HTML descargado.

Ahi ya es tarea de ustedes ingresar a cada enlace, y formatear los Skills (Tecnicos y Blancos).

:D



In [9]:

def scrape_jobs(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    jobs = []
    
    # Encontrar todos los elementos de trabajo
    job_cards = soup.find_all('div', class_='cardOutline')
    
    for job in job_cards:
        job_data = {}

        # Obtener toda la información dentro de un div con un conjunto de clases especificas
        div_info = job.find('div', class_='jobsearch-JobComponent-description')
        if div_info:
            job_data['div_info'] = div_info.get_text(strip=True)

        # Título del trabajo
        title_elem = job.find('h2', class_='jobTitle')
        if title_elem:
            job_data['title'] = title_elem.get_text(strip=True)
        
        # Nombre de la empresa
        company_elem = job.find('span', attrs={'data-testid': 'company-name'})
        if company_elem:
            job_data['company'] = company_elem.get_text(strip=True)
        
        # Ubicación
        location_elem = job.find('div', attrs={'data-testid': 'text-location'})
        if location_elem:
            job_data['location'] = location_elem.get_text(strip=True)
        
        # Salario
        salary_elem = job.find('div', attrs={'data-testid': 'attribute_snippet_testid'}, 
                            string=re.compile(r'\$'))
        if salary_elem:
            job_data['salary'] = salary_elem.get_text(strip=True)
        else:
            job_data['salary'] = "No especificado"
        
        # Requisitos (extraídos del snippet)
        requirements = []
        snippet_elem = job.find('div', attrs={'data-testid': 'belowJobSnippet'})
        if snippet_elem:
            for li in snippet_elem.find_all('li'):
                requirements.append(li.get_text(strip=True))
        job_data['requirements'] = requirements if requirements else ["No especificados"]
        
        # Enlace del trabajo
        link_elem = job.find('a', class_='jcs-JobTitle')
        if link_elem and 'href' in link_elem.attrs:
            job_data['link'] = "https://mx.indeed.com" + link_elem['href'] if not link_elem['href'].startswith('http') else link_elem['href']
        
        jobs.append(job_data)
    
    return jobs

# Ejemplo de uso (puedes reemplazar esto con tu código HTML)
if __name__ == "__main__":
    # Aquí puedes pegar tu código HTML o leerlo desde un archivo
    with open('jobs.html', 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    jobs = scrape_jobs(html_content)
    
    # Imprimir resultados
    for i, job in enumerate(jobs, 1):
        print(f"\nTrabajo #{i}")
        print(f"Título: {job.get('title', 'N/A')}")
        print(f"Empresa: {job.get('company', 'N/A')}")
        print(f"Ubicación: {job.get('location', 'N/A')}")
        print(f"Salario: {job.get('salary', 'N/A')}")
        print("Requisitos destacados:")
        for req in job.get('requirements', []):
            print(f"- {req}")
        print(f"Enlace: {job.get('link', 'N/A')}")
        print(f"Información adicional: {job.get('div_info', 'N/A')}")


Trabajo #1
Título: Analista de Financiamiento (Bancos)
Empresa: GRUPO TOMZA
Ubicación: Ciudad Juárez, Chih.
Salario: Desde $21,000 por mes
Requisitos destacados:
- Habilidad en gestión de proyectos y bases de datos.
- Perfil deAnalistade Financiamiento (Créditos Bancarios)*.
- Excelente manejo de Office (Excel, Word, etc).
Enlace: https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DWYmwa_gfd4Zu7FTm5wLV7tMEqT9N6jglDUgPcIbNj0YpxNpwqBlT8eg2rqfaVxnI9LJHxr_KivyEnJkULZZfxyLKWXdqeiHliusqulxxTLAbNlKO9r_qQatX6_I_n8JETeb6sVZwlekoRCRt18nCG4uLLgE32qiknubUzEUJEKmferOr-GMlGCrUaSj3namfmOAKX3xVdVKJ8rS-qtJMDNyhjviUsygrQTMPVFJD8me72NWUrsO3JpshwTIjveQagvV54AB_Zdey83NWe6gVu9YMWnb_kMg-_Cef7QvtJ-uGLPOy8xM5d2FGtg52qXCjEZxjy5YlByT9ysr3NK67jDv7Hq6nIMToWrO5d0FEm8H13tbUILXxuXMd_pljZKYeBlivdjYcpIubNJor8P0mr1tdIwV_v5ReiCueVW_XPHP0Hi0HJcLP7k5KOwSg_yclpig5a_czZOsAVF0YHVb0ArvUqd9beF3TS_RlwRpsEyr5zl8GM4UOBTONbe486Y_PsL4_egk1Jp5Is-2YIr8tHq4usn95R1Qu8yI-H-erTaulzNXh3QLzIB2gSaXsghHtF51VEIXcbg-GfsJXTwK2yiCCdQ6ccrrIXsB0Jz

In [10]:
# Cambiar Nombre, dependiendo el puesto
busqueda = "Analista de Datos"

In [11]:
df = pd.DataFrame(jobs)
df

,title,company,location,salary,requirements,link
0,Analista de Financiamiento (Bancos),GRUPO TOMZA,"Ciudad Juárez, Chih.","Desde $21,000 por mes",[Habilidad en gestión de proyectos y bases de ...,https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYl...
1,Analista Comercial,PUBLYCOLORES MAGIC,Home Office (Desde casa),"$12,500 por mes","[Gestión dedatos: procesar, gestionar y analiz...",https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYl...
2,Analista de Datos «Junior» «Guadalajara »,Information Technologies Consultant Home S.A. ...,Home Office (Desde casa),No especificado,[Bases deDatos: Habilidad para trabajar con ba...,https://mx.indeed.com/rc/clk?jk=5e7d9dd5b58d16...
3,ANALISTA CONTABLE,iQor,"11230, Argentina Poniente, CDMX","$9,000 a $12,000 por mes",[Elaboración de Complemento Carta Porte y/o Tr...,https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYl...
4,Analista de Base de Datos,Systeck Comunicaciones,"Home Office (Desde casa) in Ecatepec, Méx.",No especificado,[Conocimientos básicos en teoría y practica de...,https://mx.indeed.com/rc/clk?jk=e61755d0233121...
5,Analista de Datos e Información Financiera / R...,Human Talent,Home Office (Desde casa),"$15,000 a $25,000 por mes",[Consolidardatosfinancieros provenientes de di...,https://mx.indeed.com/rc/clk?jk=364b0acb67501d...
6,Analista de Soporte de Producción Jr,Qualitas Insurance,"Trabajo híbrido in Tijuana, B.C.","$24,000 por mes",[Excelente gestión de usuarios ydatossensibles...,https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYl...
7,ANALISTA DE DATOS EN TECNOLOGÍA DE LA INFORMACIÓN,MARINTER SA DE CV,"Cuauhtémoc, CDMX","$19,000 a $20,000 por mes","[Bases dedatos: MySQL, SQL (Vistas, SP y Trigg...",https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYl...
8,Analista de Datos,Tebon Class,"31165, Chihuahua, Chih.","$12,500 a $15,000 por mes",[¿Cuenta con experiencia laboral o en su carre...,https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYl...
9,Analista de Datos,Tebon Class,"64770, Monterrey, N. L.","$12,500 a $15,000 por mes","[Con marcadas habilidades numéricas, de manejo...",https://mx.indeed.com/pagead/clk?mo=r&ad=-6NYl...


In [12]:
# Exportar datos a Excel
df.to_excel(f"AI_PARSED_{busqueda.replace(' ', '_')}.xlsx", index=False, engine="openpyxl")